# MODIS SCFG

In [2]:
import xarray as xr

filename = '../data_store/data/SCFG/dap.ceda.ac.uk/2018/10/20181001-ESACCI-L3C_SNOW-SCFG-MODIS_TERRA-fv2.0.nc'

# Open the NetCDF file using xarray
data = xr.open_dataset(filename)

# Display the dataset
data.close()

<xarray.Dataset> Size: 5GB
Dimensions:      (lat: 18000, nv: 2, lon: 36000, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 8B 2018-10-01
  * lat          (lat) float64 144kB -90.0 -89.98 -89.97 ... 89.98 89.98 90.0
  * lon          (lon) float64 288kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Dimensions without coordinates: nv
Data variables:
    lat_bnds     (lat, nv) float64 288kB ...
    lon_bnds     (lon, nv) float64 576kB ...
    scfg         (time, lat, lon) float32 3GB ...
    scfg_unc     (time, lat, lon) float32 3GB ...
    spatial_ref  int32 4B ...
Attributes: (12/42)
    title:                      ESA CCI snow on ground product level L3C dail...
    institution:                ENVEO IT GmbH
    source:                     TERRA MODIS, Collection 6.1: calibrated radia...
    history:                    2021-12-10: ESA snow_cci processing line SCFG...
    references:                 http://snow-cci.enveo.at/
    tracking_id:                431e3db7-c3b7-4f43-811f-644729c31e87
    ...                         ...
    license:                    ESA CCI Data Policy: free and open access
    platform:                   TERRA
    sensor:                     MODIS
    spatial_resolution:         0.01 degree
    key_variables:              scfg
    doi:                        10.5285/8847a05eeda646a29da58b42bdf2a87c

In [9]:
import matplotlib.pyplot as plt
import numpy as np

# Define the latitude and longitude boundaries for the region of interest
lat_min, lat_max = 67.361833 - 1, 67.361833 + 1
lon_min, lon_max = 26.634154 - 1, 26.634154 + 1

# Select the subset of the data within the specified region
scfg_subset = data['scfg'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).isel(time=0)

# Set values greater than 100 (which represent flags) to NaN
scfg_subset_cleaned = scfg_subset.where(scfg_subset <= 100)

# # Plot the subset data with flag values set to NaN
# plt.figure(figsize=(10, 6))
# scfg_subset_cleaned.plot(cmap='viridis')
# plt.title(f'SCFG Data on 2018-10-01 for Sodankyla')
# plt.show()

# Define the exact latitude and longitude of interest
lat_val = 67.361833
lon_val = 26.634154

# Select the value at the specified latitude and longitude
scfg_value = scfg_subset_cleaned.sel(lat=lat_val, lon=lon_val, method='nearest').values

# Display the value
print(f"SCFG value at (lat={lat_val}, lon={lon_val}): {scfg_value}")

SCFG value at (lat=67.361833, lon=26.634154): nan


In [20]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime
import os

df=pd.read_pickle('five_sites_data_snow_cc.pkl')

# Function to convert date string from 'dd/mm/yyyy' to 'yyyymmdd'
def format_date_to_filename(date_str):
    return datetime.strptime(date_str, '%d/%m/%Y').strftime('%Y%m%d')

# Function to read the NetCDF file and extract the value at given lat/lon
def get_scfg_value(date_str, lat, lon):
    # Convert the date to a format that matches the filename
    file_date = format_date_to_filename(date_str)
    
    # Extract year and month from the date string
    year = file_date[:4]
    month = file_date[4:6]
    
    # Construct the file path
    file_path = f'../data_store/data/SCFG/dap.ceda.ac.uk/{year}/{month}/{file_date}-ESACCI-L3C_SNOW-SCFG-MODIS_TERRA-fv2.0.nc'
    
    # Check if the file exists
    if not os.path.exists(file_path):
        return np.nan  # Return NaN if the file doesn't exist
    
    # Open the NetCDF file
    dataset = xr.open_dataset(file_path)
    
    # Extract the 'scfg' variable and clean the data (set values > 100 to NaN)
    scfg_data = dataset['scfg'].where(dataset['scfg'] <= 100)
    
    # Select the nearest value for the given latitude and longitude
    try:
        scfg_value = scfg_data.sel(lat=lat, lon=lon, method='nearest').isel(time=0).values
    except KeyError:
        scfg_value = np.nan  # If lat/lon doesn't exist in the file, return NaN
    
    return scfg_value

# Function to process the dataframe row by row
def process_dataframe(df):
    scfg_values = []
    
    for index, row in df.iterrows():
        date_str = row['date']  # Extract date in 'dd/mm/yyyy' format
        lat = row['latitude']   # Extract latitude
        lon = row['longitude']  # Extract longitude
        
        # Filter out dates before October 2018 or after December 2020
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        if date_obj < datetime(2018, 10, 1) or date_obj > datetime(2020, 12, 31):
            scfg_values.append(np.nan)
        else:
            # Get the SCFG value for the current row
            scfg_value = get_scfg_value(date_str, lat, lon)
            scfg_values.append(scfg_value)
    
    # Add the SCFG values as a new column in the dataframe
    df['SCFG'] = scfg_values
    
    return df

# Assuming df is the dataframe you want to process
# Process the dataframe
df = process_dataframe(df)

# Display the updated dataframe
# df[df['SCFG'] == np.nan]
df

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


,camera,date,pvpg,y_strong,y_weak,x_strong,x_weak,longitude,latitude,meanEgstrong,...,n_photons,data_quantity,FSC,TreeSnow,JointSnow,Confidence,MxD10A1F,cc,MxD10A1,SCFG
0,sodankyla_full,12/01/2019,-0.058595,0.418798,NaN,7.147356,NaN,26.718589,67.314333,3.432135,...,566.400000,5.0,1.0,0.0,1.0,1.0,211.0,44.764267,211.0,NaN
1,sodankyla_full,12/01/2019,-0.110795,0.693207,0.281346,6.256682,2.539343,26.718589,67.319333,3.665784,...,373.625000,6.0,1.0,0.0,1.0,1.0,211.0,37.09684,211.0,NaN
2,sodankyla_full,12/01/2019,-1.307186,5.701271,1.996741,4.361483,1.527510,26.718589,67.334333,4.188318,...,469.250000,6.0,1.0,0.0,1.0,1.0,211.0,23.694944,211.0,NaN
3,sodankyla_full,12/01/2019,-0.244739,1.431021,NaN,5.847133,NaN,26.718589,67.339333,3.917523,...,681.000000,2.0,1.0,0.0,1.0,1.0,211.0,38.108734,211.0,NaN
4,sodankyla_full,12/01/2019,-0.069991,0.656128,NaN,9.374517,NaN,26.731579,67.339333,3.940102,...,698.000000,2.0,1.0,0.0,1.0,1.0,211.0,26.21748,211.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,torgnon,21/08/2023,-13.666654,3.111109,NaN,0.227642,NaN,7.542963,45.791300,0.150931,...,201.500000,2.0,0.0,0.0,0.0,1.0,0.0,None,0.0,NaN
2454,torgnon,21/08/2023,-4.053511,1.892976,NaN,0.466997,NaN,7.542963,45.796300,0.135823,...,224.000000,2.0,0.0,0.0,0.0,1.0,0.0,None,0.0,NaN
2455,torgnon,21/08/2023,-5.881129,1.735232,NaN,0.295051,NaN,7.542963,45.806300,0.106401,...,184.800000,5.0,0.0,0.0,0.0,1.0,0.0,None,0.0,NaN
2456,torgnon,21/08/2023,-76.049238,13.056713,NaN,0.171688,NaN,7.542963,45.811300,0.162791,...,157.666667,3.0,0.0,0.0,0.0,1.0,0.0,None,0.0,NaN
